In [18]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

from generate_chunks import load_data
from train_models import ModelTrainer
from failure_detection import extreme_anomaly, simple_lowpass_filter
from extract_rules import construct_features

# Load data and model
train_chunks, training_chunk_dates, test_chunks, test_chunk_dates = load_data()
with open('data/pt2_train_chunks_unnormalized.pkl', 'rb') as f:
    train_chunks_unnormalized = pickle.load(f)
with open('data/pt2_test_chunks_unnormalized.pkl', 'rb') as f:
    test_chunks_unnormalized = pickle.load(f)
model = ModelTrainer(f'configs/WAE_NOGAN.json').fit()

channel_names = ['TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs', 'Oil_temperature', 'Flowmeter', 'Motor_current', 'COMP']

# train_chunks_features = construct_features(train_chunks_unnormalized, axis=1).swapaxes(1,2)
# test_chunks_features = construct_features(test_chunks_unnormalized, axis=1).swapaxes(1,2)

# transformed_feature_names = [[fname+'_mean', fname+'_var', fname+'_max', fname+'_min'] for fname in channel_names]
# transformed_feature_names = sum(transformed_feature_names, [])

train_chunks_features = construct_features(train_chunks_unnormalized, axis=1).swapaxes(1,2)
train_chunks_features = train_chunks_features[..., [2, 3]]
test_chunks_features = construct_features(test_chunks_unnormalized, axis=1).swapaxes(1,2)
test_chunks_features = test_chunks_features[..., [2, 3]]

transformed_feature_names = [[fname+'_max', fname+'_min'] for fname in channel_names]
transformed_feature_names = sum(transformed_feature_names, [])

alpha = 0.05
threshold = 0.5

val_size = int(0.3 * len(train_chunks))
train_errors = model.calc_loss(train_chunks[-val_size:], train_chunks[-val_size:], average=False).mean(axis=(1,2))
test_errors = model.calc_loss(test_chunks, test_chunks, average=False).mean(axis=(1,2))

anom = extreme_anomaly(train_errors)
binary_output = (test_errors > anom).astype(np.int8)

output = simple_lowpass_filter(binary_output,alpha)
failures = (output >= threshold).astype(np.int8)

In [22]:

from sklearn.tree import DecisionTreeClassifier, export_text

def compare_trees(tree1, tree2):
    if tree1.get_depth() != tree2.get_depth():
        return False
    if len(tree1.tree_.feature.shape) != len(tree2.tree_.feature.shape):
        return False
    if (tree1.tree_.feature != tree2.tree_.feature).any():
        return False
    if (tree1.tree_.threshold != tree2.tree_.threshold).any():
        return False

    return True

def find_unique_trees(X, y, n=50, random_state=None):
    rng = np.random.RandomState(random_state)
    min_depth = 100
    trees = []
    for _ in range(n):
        tree = DecisionTreeClassifier(random_state=rng)
        tree.fit(X, y)
        acc = tree.score(X, y)
        if acc == 1 and tree.get_depth() <= min_depth:
            if not any([compare_trees(tree, other_tree) for other_tree in trees]) or len(trees) == 0:
                min_depth = tree.get_depth()
                trees.append(tree)
    return trees

# History stores the "good" examples assumed to be non-anomalous
history = [train_chunks_features]

# Buffer stores the datapoints during warning
buffer = []

trees = []

warning_thresh = 0.0
failure_thresh = 0.5

for t in range(len(test_chunk_dates)): 
    is_warning = output[t] > warning_thresh
    is_failure = output[t] > failure_thresh

    if is_warning:
        buffer.append(np.expand_dims(test_chunks_features[t], 0))
    else:
        # Reset buffer and add to history
        buffer = []
        history.append(np.expand_dims(test_chunks_features[t], 0))

    if is_failure and output[t-1] < output[t]:
        X_good = np.concatenate(history)
        y_good = np.zeros((X_good.shape[0]))
        X_anom = np.concatenate(buffer)
        y_anom = np.ones((X_anom.shape[0]))
        _x, _y = np.concatenate([X_good, X_anom]), np.concatenate([y_good, y_anom])
        _x = _x.reshape(_x.shape[0], -1)

        # See if any rule still applies
        new_trees = []
        for tree in trees:
            applies = tree.score(_x, _y) == 1
            if applies:
                new_trees.append(tree)
        
        # TODO: When retrianing, we get "tighter" rules (i.e., a smaller max value that still splits perfectly). Kinda unsure what to do here?
        changed = False
        if len(new_trees) != len(trees):
            changed = True
            print(f'removed {len(trees)-len(new_trees)} because the rules did not apply anymore')
        
        trees = new_trees
        if len(trees) == 0:
            changed = True
            trees = find_unique_trees(_x, _y, random_state=182616)


        print(t, output[t])
        if changed:
            for tree in trees:
                print(export_text(tree, feature_names=transformed_feature_names))
        print('---')
    else:
        #print(t, 'Reset trees and buffer')
        buffer = []
        trees = []


902 0.5123250208844702


ValueError: feature_names must contain 18 elements, got 16

In [23]:

## Restrict to not use Flowmeter
feature_indices = np.array([0, 1, 2, 3, 4, 5, 7, 8])

# History stores the "good" examples assumed to be non-anomalous
print(train_chunks_features.shape)
history = [train_chunks_features[:, feature_indices]]

# Buffer stores the datapoints during warning
buffer = []

trees = []

warning_thresh = 0.0
failure_thresh = 0.5

for t in range(len(test_chunk_dates)): 
    is_warning = output[t] > warning_thresh
    is_failure = output[t] > failure_thresh

    if is_warning:
        buffer.append(np.expand_dims(test_chunks_features[t, feature_indices], 0))
    else:
        # Reset buffer and add to history
        buffer = []
        history.append(np.expand_dims(test_chunks_features[t, feature_indices], 0))

    if is_failure and output[t-1] < output[t]:
        X_good = np.concatenate(history)
        y_good = np.zeros((X_good.shape[0]))
        X_anom = np.concatenate(buffer)
        y_anom = np.ones((X_anom.shape[0]))
        _x, _y = np.concatenate([X_good, X_anom]), np.concatenate([y_good, y_anom])
        _x = _x.reshape(_x.shape[0], -1)

        # See if any rule still applies
        new_trees = []
        for tree in trees:
            applies = tree.score(_x, _y) == 1
            if applies:
                new_trees.append(tree)
        
        # TODO: When retrianing, we get "tighter" rules (i.e., a smaller max value that still splits perfectly). Kinda unsure what to do here?
        changed = False
        if len(new_trees) != len(trees):
            changed = True
            print(f'removed {len(trees)-len(new_trees)} because the rules did not apply anymore')
        
        trees = new_trees
        if len(trees) == 0:
            changed = True
            trees = find_unique_trees(_x, _y, random_state=182616)


        print(t, output[t])
        if changed:
            for tree in trees:
                print(export_text(tree, feature_names=[tfn for tfn in transformed_feature_names if 'Flowmeter' not in tfn]))
        print('---')
    else:
        #print(t, 'Reset trees and buffer')
        buffer = []
        trees = []



(43237, 9, 2)
902 0.5123250208844702
|--- Motor_current_min <= 0.01
|   |--- class: 1.0
|--- Motor_current_min >  0.01
|   |--- class: 0.0

|--- Oil_temperature_max <= 74.44
|   |--- class: 0.0
|--- Oil_temperature_max >  74.44
|   |--- class: 1.0

|--- DV_pressure_max <= 7.24
|   |--- class: 0.0
|--- DV_pressure_max >  7.24
|   |--- class: 1.0

|--- Oil_temperature_min <= 63.77
|   |--- class: 0.0
|--- Oil_temperature_min >  63.77
|   |--- class: 1.0

---
903 0.5367087698402466
---
904 0.5598733313482342
---
removed 1 because the rules did not apply anymore
905 0.5818796647808225
|--- Motor_current_min <= 0.01
|   |--- class: 1.0
|--- Motor_current_min >  0.01
|   |--- class: 0.0

|--- Oil_temperature_max <= 74.44
|   |--- class: 0.0
|--- Oil_temperature_max >  74.44
|   |--- class: 1.0

|--- Oil_temperature_min <= 63.77
|   |--- class: 0.0
|--- Oil_temperature_min >  63.77
|   |--- class: 1.0

---
906 0.6027856815417814
---
907 0.6226463974646923
---
908 0.6415140775914577
---
909 0.

In [83]:
import pickle
from sklearn.tree import export_text

def extract_paths(tree, feature_names=None, target_idx=None):
    # Initialize a list to store all paths
    all_paths = []
    
    # Recursively traverse the tree to extract paths
    def traverse(node, path):
        # If we reach a leaf node
        if tree.tree_.feature[node] == -2:  # -2 is the indicator of a leaf node
            leaf_value = tree.tree_.value[node].tolist()[0][1]
            if target_idx is None or (target_idx is not None and int(leaf_value)) == target_idx:
                all_paths.append((path, leaf_value))
            return
        
        # Get the current feature and threshold
        if feature_names is not None:
            feature = feature_names[tree.tree_.feature[node]]
        else:
            feature = tree.tree_.feature[node]
        threshold = tree.tree_.threshold[node]
        
        # Traverse the left subtree
        left_path = path.copy()
        left_path.append((feature, threshold, '<='))
        traverse(tree.tree_.children_left[node], left_path)
        
        # Traverse the right subtree
        right_path = path.copy()
        right_path.append((feature, threshold, '>'))
        traverse(tree.tree_.children_right[node], right_path)
    
    # Start traversal from the root node
    traverse(0, [])
    
    return all_paths

def rules_to_string(paths):
    for path in paths:
        rule = path[0]
        pred = path[1]
        rule = [f'{feature} {sign} {value:.2f}' for (feature, value, sign) in rule]
        assert int(pred) == 1

        print('IF', ' AND '.join(rule), 'THEN', 'FAILURE')

channel_names = ['TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs', 'Oil_temperature', 'Flowmeter', 'Motor_current', 'COMP']

transformed_feature_names = [[fname+'_max', fname+'_min'] for fname in channel_names]
transformed_feature_names = sum(transformed_feature_names, [])

with open('d1_tree.pkl', 'rb') as f:
    d1 = pickle.load(f)
with open('d2_tree.pkl', 'rb') as f:
    d2 = pickle.load(f)

print(export_text(d1, feature_names=transformed_feature_names))
paths = extract_paths(d1, target_idx=1, feature_names=transformed_feature_names)
rules_to_string(paths)

print('---')
print(export_text(d2, spacing=1, feature_names=transformed_feature_names))
paths = extract_paths(d2, target_idx=1, feature_names=transformed_feature_names)
rules_to_string(paths)


|--- Flowmeter_max <= 16.26
|   |--- class: 0.0
|--- Flowmeter_max >  16.26
|   |--- class: 1.0

IF Flowmeter_max > 16.26 THEN FAILURE
---
|- Oil_temperature_max <= 44.06
| |- Reservoirs_min <= 8.01
| | |- class: 1.0
| |- Reservoirs_min >  8.01
| | |- class: 0.0
|- Oil_temperature_max >  44.06
| |- class: 0.0

IF Oil_temperature_max <= 44.06 AND Reservoirs_min <= 8.01 THEN FAILURE
